### Import all the libraries we wil use

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as web
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler

### Looking into our data and dropping columns we wont need as they have constant numbers or just too little change

In [2]:
df = pd.read_excel('FinalCleanedv2.xlsx',sheet_name='Petrol')
df.head()

,Date,Total Price,BFP,Fuel tax,Customes &excise,Road accident Fund,Transport cost,Petroleum Products Levy,Wholesale margin,Other Cost(BFP+FT+RAF-TP),Price per Barrel(USD),USD-ZAR,Price per Barel(Rand)
0,2011-01-01,8.58,4.53949,1.675,0.04,0.72,0.155,0.0015,0.54101,1.64551,89.17,6.9119,616.334123
1,2011-02-01,8.84,4.79949,1.675,0.04,0.72,0.155,0.0015,0.54101,1.64551,88.58,7.2032,638.059456
2,2011-03-01,9.27,5.22949,1.675,0.04,0.72,0.155,0.0015,0.54101,1.64551,102.86,6.9351,713.344386
3,2011-04-01,9.80,5.52149,1.775,0.04,0.80,0.229,0.0015,0.54101,1.70351,109.53,6.7410,738.341730
4,2011-05-01,10.09,5.81149,1.775,0.04,0.80,0.229,0.0015,0.54101,1.70351,100.90,6.8613,692.305170


In [3]:
df.shape

(129, 13)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Date                        129 non-null    datetime64[ns]
 1   Total Price                 129 non-null    float64       
 2   BFP                         129 non-null    float64       
 3   Fuel tax                    129 non-null    float64       
 4   Customes &excise            129 non-null    float64       
 5   Road accident Fund          129 non-null    float64       
 6   Transport cost              129 non-null    float64       
 7   Petroleum Products Levy     128 non-null    float64       
 8   Wholesale margin            129 non-null    float64       
 9   Other Cost(BFP+FT+RAF-TP)   129 non-null    float64       
 10  Price per Barrel(USD)       129 non-null    float64       
 11  USD-ZAR                     129 non-null    float64       

In [5]:
# Checking to see if our data is complete and in right format 
df.describe()

,Total Price,BFP,Fuel tax,Customes &excise,Road accident Fund,Transport cost,Petroleum Products Levy,Wholesale margin,Other Cost(BFP+FT+RAF-TP),Price per Barrel(USD),USD-ZAR,Price per Barel(Rand)
count,129.000000,129.000000,129.000000,1.290000e+02,129.000000,129.000000,128.000000,129.000000,129.000000,129.000000,129.000000,129.000000
mean,13.556434,6.454662,2.771047,4.000000e-02,1.454884,0.405093,0.002527,0.401528,2.876643,68.152713,12.271593,778.543680
std,2.102759,1.106942,0.719843,2.786378e-17,0.484057,0.142898,0.000895,0.093517,1.380518,23.268066,2.912485,161.100697
min,8.580000,2.747700,1.675000,4.000000e-02,0.720000,0.155000,0.001500,0.310000,0.051700,16.550000,6.741000,307.548650
25%,12.350000,5.651490,2.125000,4.000000e-02,0.960000,0.289000,0.001500,0.335000,1.863510,49.520000,10.045100,653.928165
50%,13.400000,6.325500,2.850000,4.000000e-02,1.540000,0.410000,0.003300,0.356000,2.648300,61.720000,13.158900,761.714937
75%,14.860000,7.231700,3.370000,4.000000e-02,1.930000,0.517000,0.003300,0.525000,3.525100,93.210000,14.437900,876.884390
max,18.340000,8.627900,3.930000,4.000000e-02,2.180000,0.649000,0.003300,0.580000,7.298300,109.530000,18.583000,1129.985306


In [6]:
#Create a new dataframe with only the Total Price column
X1 = df.drop(['Date'], axis=1)
X2 = X1.drop(['Customes &excise '],axis=1)
X3 = X2.drop(['Price per Barrel(USD)'], axis=1)
X4 = X3.drop(['Wholesale margin'], axis=1)
X5 = X4.drop(['Petroleum Products Levy '], axis=1)
#X = X5.drop(['Total Price'], axis=1)
data = X5.filter(['Total Price'])

In [7]:
#Convert the dataframe to a numpy array
dataset = data.values

#Get the number of rows to train the model on (80%)
training_data_len = math.ceil(len(dataset)*.8)

#Feature scaling
scale = MinMaxScaler(feature_range = (0,1))
scaled_data = scale.fit_transform(dataset)

#create the scaled training set
train_data = scaled_data[0:training_data_len,:]

In [8]:
#split data into x_train and y_train
#creating a data structure with 95 timesteps and 1 output 
#(95 timesteps is the previous 95 fuel prices and then trying to predict the next one)
X_train = []
y_train = []
for i in range(95,len(train_data)):
    #the values before that 95th value
    X_train.append(train_data[i-95:i,0])
    #the 60th value 
    y_train.append(train_data[i,0])

#convert them to numpy arays
X_train, y_train = np.array(X_train), np.array(y_train)

#Reshaping (adding a dimension in a numpy array)
#from keras documentation          Batch_size,    timesteps,    imput_dim
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1 ))
X_train.shape

(9, 95, 1)

In [9]:

#--Summary: The code in this section will build the model and train it on the training set

regressor = Sequential()

#Adding the first LSTM layer 
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1],1)))

#Adding a second LSTM layer 
regressor.add(LSTM(units = 50, return_sequences = False))

#Adding a network of neurons
regressor.add(Dense(units = 25))

#adding the output layer
regressor.add(Dense(units = 1))

#Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

#Fitting the RNN to the training set
regressor.fit(X_train, y_train, epochs = 1, batch_size = 1) #this takes long


9/9 [==============================] - 2s 24ms/step - loss: 0.2085


In [10]:
############################# -Part 3 - Predicting Next Months Petrol Price- ################################################################################################

#--Summary: The code in this section will predict te follwing month petrol price using our trained model

quote = pd.read_excel('FinalCleanedv2.xlsx',sheet_name='Petrol')
#Create a new dataframe with only the Total Price column
new_df = quote.filter(['Total Price'])

#The dataframe only keeps the last 95 months worth of data
last_95_months = new_df[-95:].values
last_95_months_scaled = scale.transform(last_95_months)

#This chunk of code gets the data ready to be put in the model
x_test = []

x_test.append(last_95_months_scaled)
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1],1 ))

#Predicts the follwing month price given the last 95 months worth of data and our model
pred_price = regressor.predict(x_test)
pred_price = scale.inverse_transform(pred_price)

#what the predicted price will be the next day
print('Petrol Price for next month is ' + str(pred_price))

Petrol Price for next month is [[20.156418]]
